# FASTGenomics Scanpy + R Analysis

<font color="red">You might want to describe your analysis briefly here, if you are planning to share it.</font>

In [ ]:
# Place all your Python imports here.
import logging
import fgread
import scanpy as sc
import scipy.sparse as spsp

# do not delete these imports as they are required for R support
import rpy2.rinterface_lib.callbacks
from rpy2.robjects import pandas2ri
import anndata2ri
%load_ext rpy2.ipython

In [ ]:
%%R
# Place all your R library imports here
suppressPackageStartupMessages({
    library(scran)
})

In [ ]:
# Place all your parameter values here.

sc.settings.verbosity = 1                                    # scanpy verbosity: errors (0), warnings (1), info (2), hints (3)
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR) # Ignore R warning messages

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()

## Raw Data
First, the raw dataset(s) will be read into an AnnData object(s).

<font color="red">You can describe your data here using markdown or delete this text.</font>

In [ ]:
# Print metadata of all attached datasets
ds_info = fgread.ds_info()
ds_info

In [ ]:
# Load the attached dataset
fgread.load_data() # If multiple datasets are attached, you have to select one by its id or tile

In [ ]:
# If you only have one dataset in your analysis you can load it by
#adata = fgread.load_data()
# You can also load a specific dataset by providing its ID or title as a parameter

## Preprocessing
<font color="red">You can describe your preprocessing here or delete this text.</font>

If this is your first analysis, you might want to have a look at our tutorials on
<a href="https://beta.fastgenomics.org/webclient/ui/#/analyses/detail-analysis-42e3571101184ed78856cbd83adfeb7e" target="_blank">Getting Started with FASTGenomics Lab</a>,
the data loading (<a href="https://beta.fastgenomics.org/webclient/ui/#/analyses/detail-analysis-9975f546d76341ea85a6c5b0870654a1" target="_blank">How to Load Data in FASTGenomics (Python)</a>),
Scanpy with R support (<a href="https://beta.fastgenomics.org/webclient/ui/#/analyses/detail-analysis-2bd51db2cb564ca7bc8dc3a9caa6b09b" target="_blank">Advanced Scanpy with R Support (rpy2)</a>), or the
<a href="https://beta.fastgenomics.org/webclient/ui/#/analyses/detail-analysis-4550a1215f5747f88ac0f83ab96d2114" target="_blank">Best Practices Preprocessing Notebook</a>.

In [ ]:
# This is an example of how to prepare AnnData matrices for processing with R
if spsp.issparse(data_all[title].X):
    data = data_all[title].X.T.todense() # if X in anndata is sparse
else:
    data = data_all[title].X.T # if X in anndata is already dense

In [ ]:
%%R -i data -o clusters
# You can use R code in cells with specified inputs and outputs
clusters <- quickCluster(data)

In [ ]:
# The outputs are then available in Python
clusters